In [ ]:
!pip install qrcode Augmentor > /dev/null
import tensorflow as tf
import matplotlib.pyplot as plt
import os, zipfile, qrcode, math, base64, IPython, Augmentor
from PIL import Image

In [ ]:
!rm -rf data/val2017 val2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip val2017.zip -d data > /dev/null

In [ ]:
!rm -rf raw
if not os.path.exists("data.zip"):
  raise Exception("Please upload a data.zip file containing your images")

with zipfile.ZipFile("data.zip", "r") as zip_obj:
  images = zip_obj.namelist()
  images = [x for x in images if x.endswith(".jpg") or x.endswith(".png") ]
  images = [ x for x in images if not "__MACOSX" in x ]
  if len(images) < 25:
    raise Exception("Please make sure data.zip contains at least 25 images (jpg or png)")
  for image in images:
    zip_obj.extract(image, f"raw")
    data = Image.open(f"raw/{image}")
    data.thumbnail([512, 512], Image.LANCZOS)
    data.save(f"raw/{image}", "JPEG")

In [ ]:
!rm -rf data/output
p = Augmentor.Pipeline("raw", output_directory="../data/output")
p.rotate90(probability=0.5)
p.rotate270(probability=0.5)
p.flip_left_right(probability=0.8)
p.flip_top_bottom(probability=0.3)
p.crop_random(probability=1, percentage_area=0.5)
p.resize(probability=1.0, width=120, height=120)
p.sample(5000)

In [ ]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_gen = data_gen.flow_from_directory(
    "data",
    target_size=(64, 64),
    batch_size=128,
    subset="training"
)

val_gen = data_gen.flow_from_directory(
    "data",
    target_size=(64, 64),
    batch_size=128,
    subset="validation"
)

In [ ]:
plt.figure(figsize=(10,10))
batch = train_gen.next()
for i in range(25):
  plt.subplot(5, 5, i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(batch[0][i], cmap=plt.cm.binary)
  plt.xlabel(batch[1][i])
plt.show()

In [ ]:
# base = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
# pool_layer = tf.keras.layers.GlobalAveragePooling2D()
# predict_layer = tf.keras.layers.Dense(train_gen.num_classes, activation="softmax")
# model = tf.keras.Sequential([base, pool_layer, predict_layer])
# model.summary()


model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(64, 64, 3)))
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), activation="relu"))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(5, 5), activation="relu"))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(train_gen.num_classes, activation="softmax"))
model.summary()


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.categorical_crossentropy
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

model.fit(
    train_gen,
    epochs=25,
    steps_per_epoch=25,
    validation_data=val_gen,
    validation_steps=5
)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [ tf.lite.Optimize.OPTIMIZE_FOR_SIZE ]
model_data = converter.convert()

base64_bytes = base64.b64encode(model_data).decode("ascii")
num_slides = math.ceil(len(base64_bytes) / 2300)

imgs = []
for n in range(num_slides):
  start = n*2300
  end = min(start + 2300, len(base64_bytes))
  qr = qrcode.QRCode(version=40)
  qr.add_data(f"{n},{num_slides},{base64_bytes[start:end]}")
  img = qr.make_image(fill="black", back_color="white")
  imgs += [ img.resize((1024, 1024)).convert("P") ]

imgs[0].save("qr.gif", format="GIF", append_images=imgs[1:], save_all=True, duration=100, loop=0)
print("Scan the folowing QR code with the MLClassifier app:")
IPython.display.Image(open("qr.gif","rb").read(), width=512, height=512)